In [1]:
f=open("./药典/一捻金.txt")

In [7]:

line=""
while True:
    line=f.readline()
    if "功能与主治" in line:
        break
line.strip().split("。")[1]        

In [8]:
line

'    【功能与主治】消食导滞，祛痰通便。用于脾胃不和、痰食阻滞所致的积滞，症见停食停乳、腹胀便秘、痰盛喘咳。\n'

'用于脾胃不和、痰食阻滞所致的积滞，症见停食停乳、腹胀便秘、痰盛喘咳'

In [5]:
"一" in a

True

In [18]:
import os
files= os.listdir("./药典/")

In [20]:
len(files)

4761

In [57]:
import csv
import os
import re
files=os.listdir("./药典/")

count=0
#将观察到的垃圾症状count放在这里就可以了
rubbish=[38,44,163,210,312,482,858,899,1570,1655,2098,2296,2450,2542,2507,2760,3510,3546]
with open("Extraction.csv","w") as csvfile: 
    writer = csv.writer(csvfile)
    writer.writerow(["index","药名","症状","不明症状","用法","每次用量","每日用量","症状原文","用法用量原文"])
    for file in files:
        with open("./药典/"+file,"r") as f:
            #print(count,file)
            line=""
            origin_sym=""
            origin_use=""
            while True:
                line=f.readline()
                origin_sym=line
                if not line:
                    break
                if "功能与主治" in line:
                    break
            
            #到这里分割出"【功能与主治】"行
            count+=1
            name=file.split(".")[0]
            symptom=""
            unclear_symptom=""
            #表意不清症状
            unclear=[""]
            usage=""
            dosage1=""
            dosage2=""
            
            if line !="" and line:
                #print(line.strip())
                line=line.strip().split("。")[1].replace("用于","")#此处分割后只有后面的"用于"开头的语句
                line=line.replace("甩于","")
                line=line.split("；")[0] # "；"后面是"慢性鼻炎、过敏鼻炎、鼻窦炎见上述证候者", 类似的，除去
                line=line.split("，")#症状整体使用"，"分隔，内部可能涉及有“、”分隔
                for i in range(len(line)):
                    line[i]=re.sub(r'.*所致',"",line[i]) #去除各个症状中间"所致的"部分
                    line[i]=re.sub(r'.*的',"",line[i])#去除原因
                    line[i]=line[i].replace("症见","") #去除“症见”关键词
                    line[i]=line[i].replace("或","、")#"或"表达的是并列关系
                    line[i]=line[i].replace("以及","、") #同上
                    line[i]=line[i].replace("及","、") #同上
                    line[i]=line[i].replace("和","、")
                    line[i]=line[i].replace(" ","") #去除内部空格
                    
                    line[i]=line[i].strip("、：")#去除首尾无关紧要的符号
                    
                    #以下内容是否去除,不明。单列一项，便于之后的处理                    
                    # 此症状需要上述所有症状的信息，不可单独罗列，故提取
                    if "上述证候者" in line[i] or( re.search(r'证$',line[i]) and not re.search(r"淋证$",line[i]))\
                            or re.search("者$",line[i]) \
                            or re.search(r'证者$',line[i]) \
                            or re.search(r"证候者$",line[i]) \
                            or re.search("乳房肿块",line[i]) \
                            or re.search("乳房结节",line[i]) \
                            or re.search("乳房结块",line[i]) \
                            or re.search("Ⅰ、Ⅱ",line[i]) \
                            or re.search("I、II",line[i])\
                            or re.search("【功能与主治】",line[i])\
                            or re.search("【用法与用量】口服",line[i])\
                            or re.search("带下量多",line[i]):
                            
                        unclear.append(line[i])
                    
                    
                    
                if(len(unclear)>0):
                    for un in unclear:
                        if un in line:
                            line.remove(un)
                    #print("unclear",end=" ") #显示那些标示为unclear的项
                    #print(unclear)
                                
                #第二趟扫描
                #print(line)
                comma_pos=[]
                for i in range(len(line)):
                    #由"，"分割后，有很大部分是有"、"来分割的，部分并不能去除
                    noComma=line[i].split("、")
                    if(len(noComma)>1):
                        # 分割后如果有字数为1的项，一定不合理
                        noComma_ok=True
                        for item in noComma:
                            if len(item)==1:
                                unclear.append(line[i])
                                #print("noComma-WRONG",end="") #查看去除”、“后不合理的项
                                #print(line[i])
                                noComma_ok=False
                                break
                        if noComma_ok:
                            comma_pos.append(i)
                            line.extend(noComma)
                            #line.pop(i)
                            #for item in noComma.reverse():
                            #    line.insert(i,item)
                            #line[i]=noComma
                            #print("noComma-OK",end="") #查看其他的去除"、"是否可行
                            #print(line[i])
                for i in comma_pos[::-1]:
                    line.pop(i)
                
                #第二次去除            
                if(len(unclear)>0):
                    for un in unclear:
                        if un in line:
                            line.remove(un)
                    #print("unclear",end=" ") #显示那些标示为unclear的项
                    #print(unclear)  
                #此处为筛查；可去除
                
                
                symptom=" ".join(line)
                symptom=symptom.strip().strip(".")
                unclear_symptom=" ".join(unclear)
                #print(symptom)
                #print(unclear_symptom)
                
                
                #以上均为处理【症状】
                #以下处理【用法与用量】
                line=f.readline()
                line=f.readline()
                
                
                if re.search(r"【.*用法.*】",line):
                    origin_use=line
                    line=re.sub(r"【.*用法.*】","",line).strip("。 \n")
                    if "口服" in line:
                        #usage.insert(0,"口服")
                        usage="口服"
                        line=line.replace("口服","")
                    elif "开水冲服" in line:
                        #usage.insert(0,"开水冲服")
                        usage="开水冲服"
                        line=line.replace("开水冲服","")
                    elif "外用" in line:
                        #usage.insert(0,"外用")
                        usage="外用"
                        line=line.replace("外用。","")
                    elif "外用适量" in line:
                        #usage.insert(0,"外用适量")
                        usage="外用适量"
                        line=line.replace("外用适量","")
                    
                    
                    line=line.replace(" ","")
                    pattern1=re.compile(r'(一次)?([0-9\.]*～[0-9\.]*|[0-9\.]*)(ml|mL|g|袋|片|丸|粒|块|揿|滴|锭|贴|枚)')
                    pattern2=re.compile(r'(—日|一日|每日)([0-9\.]*(～|~|-)[0-9\.]*|[0-9\.]*)次')
                    result1=pattern1.findall(line)
                    line=re.sub(pattern1,"",line)
                    result2=pattern2.findall(line)
                    line=re.sub(pattern2,"",line)                    
                    
                    #print(usage)
                    #print(result1,end="")
                    #print(result2,end="")
                    line=line.strip("。，；")
                    #print(line)
                    #if len(result1)==0 and len(result2)==0:
                    #    print(line)
                    
                    #writer.writerow(["index","药名","症状","不明症状","用法","每次用量","每日用量","症状原文","用法用量原文"])
                    if(len(result1)!=0):
                        for item in result1:
                            if item[1]!="":
                                dosage1=dosage1+"".join(item)+";"
                    
                    if(len(result2)!=0):
                        for item in result2:
                            if item[1]!="":
                                dosage2=dosage2+"".join(item[0:2])+"次;"    
                        
                    
            if count in rubbish:
                writer.writerow([count,name,"",symptom+unclear_symptom,usage,dosage1,dosage2,origin_sym,origin_use])
            else:
                writer.writerow([count,name,symptom,unclear_symptom,usage,dosage1,dosage2,origin_sym,origin_use])
            if symptom!="":
                print(count,symptom)
            #if unclear_symptom!="":
            #    print(count,unclear_symptom)            
            #print(count,dosage1,dosage2)        
                
            #print(count,name,content)
            #writer.writerow([count,name,content,usage])
            

1 热病神昏 中风痰迷 小儿痰热惊痫 抽搐 夜啼
7 咳嗽 气促发喘 咯吐白痰 畏寒肢冷 腰痠背冷 腹胀食少
9 瘀血经闭 癥瘕积聚 虫积腹痛
10 易感冒 鼻痒 喷嚏 流清涕
13 鼻塞 鼻痒气热 流涕黄稠 持续鼻塞 嗅觉迟钝
14 眩晕耳鸣 呕吐 噫气 呃逆 喘息 吐血 齟血 崩漏下血
15 小儿风热感冒 发热重 头胀痛 咳嗽痰黏 咽喉肿痛
17 淋证 水肿 心烦尿赤 口舌生疮 经闭乳少 湿热痹痛
19 头晕目眩 头痛失眠 烦躁不安
23 咳嗽气喘 胸满痰多 肠燥便秘
24 上消化道出血量多 烦躁 神志淡漠 肢冷 汗出 脉弱无力
28 泄泻 大便稀薄如水样 腹痛 纳差
30 风热感冒 发热 周身不适 头痛 目眩 咽喉肿痛
31 白癜风 白斑散在分布 色泽苍白 边界较明显
32 大便出血 血色鲜红 肛门灼热疼痛
33 感冒 头痛昏重 胸膈痞闷 脘腹胀痛 呕吐泄泻
34 鼻塞 流清涕 浊涕 前额头痛
35 风湿性关节痛 关节扭伤
37 经闭 月经不调 难产 胎盘不下 产后瘀血腹痛
38 月经不调 带下病 月经先后不定期 量多 淋漓不净 色淡无块
42 暴发火眼 红肿作痛 头晕目眩 眼边刺痒 大便燥结 小便赤黄
43 胃痛 胁痛 头痛 痛经
44 油风 毛发成片脱落 至全部脱落 多伴有头晕失眠 目眩耳鸣 腰膝痠软
47 肾阴亏损 头晕耳鸣 腰膝酸软 骨蒸潮热 盗汗遗精 消渴
49 痨嗽咯血 跌扑损伤 咳嗽痰多 病后虚弱
50 失眠 多梦 头晕
51 膏淋 白浊 白带过多 风湿痹痛 关节不利 腰膝疼痛
54 胸痹 心痛 眩晕
56 外寒内热 表里俱实 恶寒壮热 头痛咽干 小便短赤 大便秘结 瘰疬初起 风疹湿疮
60 风湿痛 跌打损伤 肿痛
63 咳嗽气喘 咯痰色白
65 尪痹 局部肿大 僵硬畸形 屈伸不利 腰膝痠软 畏寒乏力 肌肉 关节疼痛
66 肝区胀痛 胸闷不舒 食欲不振
68 外伤骨折 筋骨扭伤 刀口疼痛 出血 风湿痹痛 崩漏 痛经 牙龈肿痛 出血
70 脘腹胀满 食欲不振 呕吐
72 热病 邪入心包 高热惊厥 神昏谵语
75 便秘 咽部红肿 咽痛 发热 口渴
79 跌打损伤 筋骨折伤 瘀肿疼痛
85 便秘 湿热下痢 口渴不休 停食停水 胸热心烦 小便赤黄
89 月经不调 痛经 麻木瘫痪 风湿痹痛 气血虚弱
92 浊涕臭气 嗅觉失灵 慢性鼻窦炎 鼻炎 鼻塞头痛
9

930 胃脘灼热疼痛 呕吐吞酸 口苦嘈杂 腹痛泄泻
932 胸痹心痛 胸胁刺痛 跌扑肿痛 月经不调 经闭痛经 癥瘕腹痛 头痛 风湿痹痛
934 风热咳嗽 痰多上气 燥咳
939 外感发热头痛 项背强痛 口渴 消渴 麻疹不透 热痢 泄泻 眩晕头痛 中风偏瘫 胸痹心痛 酒毒伤中
941 口舌生疮 咽喉疼痛 心胸烦热 小便短赤 大便秘结
942 湿浊中阻 脘痞不饥 脾胃虚寒 呕吐泄泻 妊娠恶阻 胎动不安
949 胃痛 胁痛 头痛 痛经
952 痈肿疔疮 蛇虫咬伤 臌胀水肿 湿热黄疸 湿疹湿疮
954 头痛 有恶寒 发热 鼻塞
958 热病伤阴 舌绛烦渴 温毒发斑 吐血 衄血 咽喉肿痛
962 肉食积滞 瘀血癥瘕 腹中痞块 虫积腹痛
963 气虚乏力 食少便溏 中气下陷 久泻脱肛 便血崩漏 表虚自汗 气虚水肿 内热消渴 血虚萎黄 半身不遂 痹痛麻木 痈疽难溃 久溃不敛
969 痰饮呕吐 胃脘痞满
972 肝肾不足 腰膝酸痛 筋骨无力 头晕目眩 妊娠漏血 胎动不安
973 吐泻腹痛 胸闷 胁痛 头痛发热
977 腹泻 痢疾 食欲不振 内外伤出血 肺结核咳嗽 气管炎咳嗽 风湿疼痛 跌打损伤
978 失音声哑
980 失眠多梦 心悸 记忆力减退
983 月经不调 产后恶露不绝 经水量少 淋漓不净 产后出血时间过长
985 咳嗽痰多
990 两胁胀痛 隐痛 乏力 尿黄
994 脾胃湿热 胃脘胀痛 胃酸过多
995 癥块 经闭 痛经 产后恶露不尽
996 咽干 咽痛 喉核肿大 口渴 发热
997 咳嗽痰多 胸闷不畅 咽痛音哑 肺痈吐脓
998 动脉硬化症 高脂血症
1001 气虚咳嗽 暑热烦躁 津伤口渴 头目不清 四肢倦乏
1002 中风气虚血瘀 半身不遂 口舌歪斜 言语謇涩 头晕目眩 半身麻木 气短乏力
1007 热病心烦 湿热黄疸 淋证涩痛 血热吐衄 目赤肿痛 火毒疮疡
1011 肝肾两亏 风寒湿痹 腰膝冷痛 屈伸不利
1016 肝胃气滞 脘腹胀痛 胸膈痞满 不思饮食 嗳气泛酸
1017 白浊 小便频数
1019 贫血衰弱 痛经 病后 产后血虚 月经不调
1022 鼻痒 喷嚏 流清涕 易感冒
1023 中风痰厥 猝然昏倒 胸痹心痛 胸腹冷痛 惊痫
1027 口疮肿痛 疮疡溃烂 烦躁口渴 大便秘结
1028 衄血 吐血 尿血 血淋 便血 崩漏 外伤出血 痈肿疮毒
1029 风寒感冒

1883 跌打损伤 风湿瘀阻 关节痹痛
1885 痹病 头晕耳鸣 关节疼痛 腰膝痠软 畏寒肢冷 手足麻木
1886 肿瘤患者具有缓解症状 缩小瘤体 提高机体免疫力 作用
1891 中风中经络 半身不遂 偏身麻木 口眼歪斜 语言謇涩
1895 痰热咳嗽 咯痰黄稠 中风痰迷 癫狂惊痫
1897 寒疝腹痛 睾丸偏坠 痛经 少腹冷痛 脘腹胀痛 食少吐泻
1898 跌打损伤 瘀血疼痛 闪腰岔气
1901 乳癖 经前胀痛 腰痠乏力 经少色淡
1906 小便频数 夜间遗尿
1907 吐血 崩漏 便血 痔血 赤白带下 久痢不止
1909 心神不宁 失眠多梦 健忘 倦怠乏力 食欲不振
1914 风热外侵 头痛 咽痛 鼻塞流涕 咳嗽痰黏 口渴 溲黄 发热 微恶风
1918 咽干 咽痛 咽部充血
1921 咳喘 头痛鼻塞 痰多咳嗽 胸闷气喘
1926 肺虚久咳 久泻久痢 虚热消渴 蛔厥呕吐腹痛
1927 脘腹冷痛 呕吐泄泻 虫积腹痛
1929 胸痛 胸闷 心慌 气短
1932 精血亏虚 头晕眼花 耳鸣耳聋 须发早白 病后脱发 肠燥便秘
1938 黄疸尿赤 热淋涩痛
1942 痹病 关节疼痛 屈伸不利 四肢麻木
1944 胸痹 胸闷胸痛 心悸怔忡 五心烦热 夜眠不安 舌红少荅
1946 痰涎壅肺 喘咳痰多 胸胁胀满 不得平卧 胸腹水肿 小便不利
1950 吐血 衄血 外伤出血 肝郁胁痛 胸痹刺痛 跌扑伤痛 呕吐腹痛
1951 气喘咳嗽 吐痰清稀
1955 痰多喘咳 新久咳嗽 劳嗽咳血
1960 痈疽疔疮 梅毒下疳 一切恶疮 肉暗紫黑 腐肉不去 窦道瘘管 脓水淋漓 久不收口
1962 咳嗽 痰黄而黏不易咳出 胸闷气促 久咳不止 声哑喉痛
1963 疔疮肿毒 乳痈 瘰疬 目赤 咽痛 肺痈 肠痈 湿热黄疸 热淋涩痛
1964 肝火亢盛 失眠多梦 心烦
1968 心烦失眠 尿少涩痛 口舌生疮
1969 痹病 骨节疼痛 腰膝酸软 肢体麻木拘挛
1971 气血不足 心悸怔忡 健忘失眠 血虚萎黄
1973 胸痹 胸闷憋气 心前区刺痛 心悸健忘 眩晕耳鸣
1975 阳虚欲脱 冷汗自出 四肢厥逆 下利清谷 脉微欲绝
1977 胸满腹胀 头昏浮肿 寒嗽痰喘
1982 水肿胀满 二便不通
1984 温疫时毒 发热咽痛 温毒发斑 痄腮 烂喉丹痧 大头瘟疫 丹毒 痈肿
1985 郁闷不舒 胸胁胀痛 头晕目眩 食欲减

2816 久泻久痢 便血脱肛 肺虚喘咳 久嗽不止 咽痛音哑
2817 脾胃虚弱 脘腹胀满 食少便溏
2818 骨质增生 风湿性关节炎 风湿痛
2819 感冒发热 咽喉肿痛 口舌生疮 顿咳劳嗽 泄泻痢疾 热淋涩痛 痈肿疮疡 蛇虫咬伤
2821 肝风上扰 偏头痛 紧张性头痛
2822 咽部红肿 咽痛 失音声哑 口干舌燥 干咳少痰
2823 胸痹心痛 胃脘疼痛 痛经经闭 产后瘀阻 癥瘕腹痛 风湿痹痛 筋脉拘挛 跌打损伤 痈肿疮疡
2825 癫狂惊悸 喘咳痰稠 大便秘结
2827 风热感冒 发热头痛 咳嗽口干 咽喉疼痛
2828 关节僵硬 屈伸不利 畏寒肢冷 关节刺痛 钝痛
2833 表寒里热 身热口渴 咳嗽痰盛 喘促气逆 胸膈满闷
2834 目翳外障 视物不清 隐痛流泪
2837 面色萎黄 腰痛倦怠 恶心呕吐 食欲不振 小便不利 大便黏滞
2838 咽喉肿痛 肺热咯血 跌打肿痛
2839 口舌生疮 齿龈 咽喉肿痛
2840 产后气血亏损 乳少 无乳 乳汁不通
2841 风热感冒 发热头痛 咳嗽口干 咽喉疼痛
2845 食积停滞 脘腹胀满 嗳腐吞酸 不欲饮食
2847 湿阻气滞 脘腹胀闷 大便不爽 水肿胀满 脚气浮肿 小便不利
2848 肿瘤患者具有缓解症状 缩小瘤体 提高机体免疫力 作用
2849 小儿惊风 抽搐神昏
2850 风湿痹痛 水肿脚气 小便不利 湿疹疮毒
2851 痈疽初起 脓成不溃
2856 胸膈痞闷 脘腹胀痛 呕吐恶心 嗳气纳呆
2857 外治湿疹 疥癣 脱肛 痔疮 聤耳流脓
2858 行经后错 行经小腹疼痛 血块排出痛减 经水量少 有血块 经前双乳胀痛 烦躁 食欲不振
2860 口渴喜饮 易饥多食 气短乏力
2861 喉痹 乳蛾 咽喉肿痛 口苦咽干 喉核红肿
2862 缺血性中风中经络（脑梗塞恢复期）
2864 头痛如裹 目赤口渴 小便不利
2867 热性“赤巴”病 胆囊炎 黄疸型肝炎
2873 胸痹 胸痛 胸闷 心悸 气短 乏力
2877 鼻塞 流涕 发热 头痛
2879 上呼吸道感染 急性咽喉炎 急性扁桃体炎 急性肠胃炎 亦可细菌性痢疾
2880 失眠 多梦 健忘 乏力 心悸 气短 自汗
2882 热淋 石淋 尿频 尿急 尿痛 腰痛
2884 顽痰胶结 咳逆喘急 癫痫发狂 烦躁胸闷 惊风抽搐
2890 气虚血瘀 胸痹心痛 中风偏瘫 倦怠气喘
2892

3786 中老年人骨质疏松症 腰脊痠痛 胫膝痠软 神疲乏力
3789 寒湿内阻 脘腹胀痛 痞满呕吐 疟疾寒热 瘟疫发热
3791 阴虚潮热 骨蒸盗汗 肺热咳嗽 咯血 衄血 内热消渴
3793 头晕目眩 头痛失眠 烦躁不安
3794 胸痹 胸闷 心前区疼痛
3807 头痛眩晕 目赤耳鸣 咽喉肿痛 口舌生疮 牙龈肿痛 大便燥结
3808 便秘 大便不通 脘腹胀满 口苦尿黄
3811 感冒发热 喉炎 胸腹胀痛 虚劳心悸 月经不调 产后流血
3817 胸痹心痛 眩晕头痛 经期腹痛
3820 脘腹刺痛 风湿痹痛
3821 肾阳不足 腰膝痠冷 肢体浮肿 痰饮喘咳 消渴
3822 湿热下注 小便短赤 淋沥涩痛 口燥咽干
3824 中风 半身不遂 口舌歪斜 手足麻木 疼痛痉挛 言语謇涩
3825 月经过多 崩漏 吐血 衄血 咯血 尿血 便血 血色鲜红 紫红
3831 自然毒 食物毒 配制毒等各种中毒症
3832 热病神昏 中风痰迷 惊痫抽搐 癫痫发狂 咽喉肿痛 口舌生疮 痈肿疔疮
3833 阴虚火旺 虚火上浮 口鼻干燥 咽喉肿痛
3834 肺肾两虚 精气不足 久咳虚喘 神疲乏力 不寐健忘 腰膝痠软 月经不调 阳痿早泄
3836 风热感冒 发热 周身不适 头痛 目眩 咽喉肿痛
3837 急慢性病毒性肝炎 痈疽疔疮 无名肿毒 跌打损伤 各种炎症
3839 肾阳不足 阳痿遗精 遗尿尿频 腰膝冷痛 肾虚作喘 五更泄泻
3840 溃疡 疮面疼痛 疮色鲜活 脓腐将尽
3842 气粗痰涌 痰鸣如吼 咳呛阵作 痰黄稠厚
3848 咽部肿痛 口舌生疮 牙龈红肿 口干舌燥
3851 水肿 浮肿 乏力 腰膝冷痛
3852 带下病 月经不调
3853 气分热盛 高热烦躁
3855 再生障碍性贫血 白细胞减少症 血小板减少症
3857 感冒 发热恶风 头痛目赤 咽喉肿痛
3858 中风中经络 半身不遂 肢体麻木 口眼歪斜 舌强语謇 胸痹心痛 胸闷 心悸 气短
3859 脾胃气虚 胃痛 胃脘隐痛 刺痛 纳呆食少
3860 寒痰咳喘 阳虚冷喘 腰膝冷痛 胃痛泛酸 乳汁不通
3861 头晕目眩 痰涎壅盛 神志混乱 言语不清 惊风抽搐 癫痫
3864 小儿风寒感冒 停食停乳 发热鼻塞 咳嗽痰多 呕吐泄泻
3865 咳嗽 喘急气短
3870 跌打损伤 外伤出血 风湿痹痛 黄水病
3873 腹痛泄泻 泄泻 痢疾 里急后

4511 过敏性鼻炎
4513 瘿瘤 瘰疬 睾丸肿痛 痰饮水肿
4514 高脂血症 气短 乏力 头晕 头痛 胸闷 腹胀 食少纳呆
4516 小儿外感发热
4517 关节肿胀 麻木 疼痛 活动受限
4520 经闭 痛经 腰膝酸痛 筋骨无力 淋证 水肿 头痛 眩晕 牙痛 口疮 吐血 衄血
4521 肝肾阴虚 牙齿松动 须发早白 眩晕耳鸣 腰膝酸软 血痢 崩漏下血 外伤出血 阴虚血热吐血 衄血 尿血
4525 月经不调 经闭痛经 目赤翳障 头晕胀痛
4526 遗精滑精 遗尿尿频 脾虚久泻 白浊 带下
4530 消渴病 多食 多饮 多尿 消痩 四肢无力
4538 胸胁胀满 胃脘疼痛 嗳气呕恶 食少纳呆
4539 蛔虫病 蛲虫病 绦虫病 虫积腹痛 小儿疳积
4543 妇女癥瘕 小腹胀痛 经色紫黯有块 经行不爽
4544 癥瘕积聚 风寒湿痹 胃寒疼痛 手足麻木
4548 眩晕耳鸣 腰膝酸痛 阳痿遗精 遗尿尿频 崩漏带下 大汗虚脱 内热消渴
4552 湿热黄疸 胁痛 痢疾腹痛 食欲不振
4558 感冒 发热 咳嗽 咽痛
4560 顽痰胶结 咳逆喘急 癫痫发狂 烦躁胸闷 惊风抽搐
4561 风湿痹痛 筋骨无力 腰膝酸软 四肢麻痹 半身不遂 风疹湿疮
4562 脱发症
4567 头晕 失眠 胸闷 胸痛 心悸 气短 乏力
4577 胃脘刺痛 胸腹胀满 恶心呕吐 停食停水 气滞闷郁
4578 便血 肛门肿痛 有下坠感
4579 心神不安 忧郁失眠 肺痈 疮肿 跌扑伤痛
4581 气阴两虚 心悸不宁 气短乏力 胸闷胸痛
4583 咽部肿痛 咽干少津 咽部白腐有苔膜 喉核肿大
4585 寒湿闭阻 瘀血阻络 痹病 关节冷痛 屈伸不利
4590 肝肾阴虚 耳鸣耳聋 头晕目眩
4591 月经不调 痛经 经期后错 经血量少 行经腹痛
4594 附骨疽 发热 口渴 局部红肿 疼痛 流脓
4597 目赤肿痛 睑弦赤烂 翳膜遮睛 胬肉攀睛 溃疡不敛 脓水淋漓 湿疮瘙痒
4602 肝胃气痛 食少呕恶 月经不调 跌扑伤痛
4603 风寒湿痹 腰膝冷痛 拘挛麻木 筋骨痿软
4605 头痛头昏 恶心呕吐 视物不清 肢体麻木 耳鸣
4607 湿热内停 赤白痢疾 里急后重 胃肠积滞 脘腹胀痛 大便不通
4612 心肺火盛 胸闷不舒 胸胁闷痛 心悸气短
4614 感冒 恶寒发热 头痛鼻塞 咳嗽痰多 胸闷呕逆 乏力气短


In [25]:
a=['眩晕、中风', '头晕目眩、烦躁不宁、言语不清、手足不遂']
no=[]
for i in range(len(a)):
    temp=a[i].split("、")
    no.append(i)
    a.extend(temp)
    print(temp)

for i in no[::-1]:
    a.pop(i)
a

['眩晕', '中风']
['头晕目眩', '烦躁不宁', '言语不清', '手足不遂']


['眩晕', '中风', '头晕目眩', '烦躁不宁', '言语不清', '手足不遂']

In [210]:

a="一次1.2～2g 一次3g"
p=re.compile(r"一次([0-9\.]*～[0-9\.]*|[0-9\.]*)")
r=p.findall(a)
r

['1.2～2', '3']

In [24]:
a=[1,2,3]
type(a.reverse())

NoneType

In [84]:
import re
a="火热毒盛所致的热疖痈毒、咽喉肿痛、风热火眼"
a=re.sub(r'.*所致的',"",a)
a

'热疖痈毒、咽喉肿痛、风热火眼'

In [80]:
import re
 
phone = "2004-959-559 # 这是一个国外电话号码"
 
# 删除字符串中的 Python注释 
num = re.sub(r'#.*$', "", phone)
print("电话号码是: ", num)
 
# 删除非数字(-)的字符串 
num = re.sub(r'\D', "", phone)
print("电话号码是 : ", num)

电话号码是:  2004-959-559 
电话号码是 :  2004959559


In [8]:
a="、q、q："
a.strip("、：")

'q、q'